In [17]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

import keras as k
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint

import cv2
import tifffile as tiff
from tqdm import tqdm

x_train = []
x_test = []
y_train = []

df_train = pd.read_csv('../input/amazon/train.csv')

flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

for f, tags in tqdm(df_train.values, miniters=1000):
    img = tiff.imread('../input/amazon/train-tif/{}.tif'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train.append(cv2.resize(img, (32, 32)))
    y_train.append(targets)
    
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float16) / 255.

print(x_train.shape)
print(y_train.shape)

# split = 35000
split = 80
x_train, x_valid, y_train, y_valid = x_train[:split], x_train[split:], y_train[:split], y_train[split:]

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(32, 32, 4)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))

model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
              optimizer='adam',
              metrics=['accuracy'])
              
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          callbacks=[ModelCheckpoint('keras-simple.model', monitor='val_loss', verbose=0, mode='auto', period=1)],
          validation_data=(x_valid, y_valid))
          
p_valid = model.predict(x_valid, batch_size=128)
print(y_valid)
print(p_valid)
                
from sklearn.metrics import fbeta_score
def f2_score(y_true, y_pred):
    # fbeta_score throws a confusing error if inputs are not numpy arrays
    y_true, y_pred, = np.array(y_true), np.array(y_pred)
    # We need to use average='samples' here, any other average method will generate bogus results
    return fbeta_score(y_true, y_pred, beta=2, average='samples')
                     
print(f2_score(y_valid, np.array(p_valid) > 0.2))

100%|██████████| 100/100 [00:00<00:00, 143.84it/s]


(100, 32, 32, 4)
(100, 17)
Train on 80 samples, validate on 20 samples
Epoch 1/20
80/80 [==============================] - 2s - loss: 2.7319 - acc: 0.5169 - val_loss: 1.9715 - val_acc: 0.7794
Epoch 2/20
80/80 [==============================] - 0s - loss: 2.7753 - acc: 0.7213 - val_loss: 0.9994 - val_acc: 0.8912
Epoch 3/20
80/80 [==============================] - 0s - loss: 1.8996 - acc: 0.8044 - val_loss: 1.0092 - val_acc: 0.9000
Epoch 4/20
80/80 [==============================] - 0s - loss: 1.4499 - acc: 0.8463 - val_loss: 1.0416 - val_acc: 0.9000
Epoch 5/20
80/80 [==============================] - 0s - loss: 1.3659 - acc: 0.8419 - val_loss: 0.8557 - val_acc: 0.9000
Epoch 6/20
80/80 [==============================] - 1s - loss: 1.1565 - acc: 0.8382 - val_loss: 0.6083 - val_acc: 0.9000
Epoch 7/20
80/80 [==============================] - 0s - loss: 0.8466 - acc: 0.8507 - val_loss: 0.3654 - val_acc: 0.9000
Epoch 8/20
80/80 [==============================] - 0s - loss: 0.5036 - acc: 0.850